<a href="https://colab.research.google.com/github/hemsmalli5/Final-Project---Week1/blob/hemsmalli5/Project_Database_Connections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark


# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-2.4.7-bin-hadoop2.7"


# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,104 kB]
Get:15 ht

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-10-08 08:03:02--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.02MB/s    in 0.9s    

2020-10-08 08:03:04 (1.02 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/title.akas.tsv"
spark.sparkContext.addFile(url)
movie_akas = spark.read.csv(SparkFiles.get("title.akas.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
movie_akas.show(10)

+---------+--------+--------------------+------+--------+-----------+----------+---------------+
|  titleId|ordering|               title|region|language|      types|attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+----------+---------------+
|tt0000001|       1|Carmencita - span...|    HU|      \N|imdbDisplay|        \N|              0|
|tt0000001|       2|          Καρμενσίτα|    GR|      \N|         \N|        \N|              0|
|tt0000001|       3|          Карменсита|    RU|      \N|         \N|        \N|              0|
|tt0000001|       4|          Carmencita|    US|      \N|         \N|        \N|              0|
|tt0000001|       5|          Carmencita|    \N|      \N|   original|        \N|              1|
|tt0000002|       1|Le clown et ses c...|    \N|      \N|   original|        \N|              1|
|tt0000002|       2|   A bohóc és kutyái|    HU|      \N|imdbDisplay|        \N|              0|
|tt0000002|       3|Le clown e

In [5]:
# Check length of the Data
movie_akas.count()

19549634

### Notes

Due to the format of the akas file, all rows that do not have US as the region and all movies that have duplicate rows will need to be dropped in SQL before it can be merged with the other datasets. All other ETL steps can take place using Python in the Jupter notebook / CoLab file

--OR--

Save movie_akas_us as tsv file and load that into SQL


In [19]:
#Make copy of df for editing
movie_akas_us = movie_akas.copy()

AttributeError: ignored

In [ ]:
# Drop all rows that do not have region as US
movie_akas_us = (movie_akas_us.loc[movie_akas_us['region'] == 'US'])

# Drop all rows where types is alternative
movie_akas_us = movie_akas_us[movie_akas_us.types != 'alternative']

# Drop duplicate rows if there is more then one row per movie (keep first row)
movie_akas_us = movie_akas_us.drop_duplicates(subset=['tconst'], keep='first')
movie_akas_us

In [6]:
# Read in title_basics data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/title.basics.tsv"
spark.sparkContext.addFile(url)
movie_basics = spark.read.csv(SparkFiles.get("title.basics.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
movie_basics.show(10)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            \N|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [7]:
# Check length of the Data
movie_basics.count()

6326545

In [8]:
# Read in title_akas data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/title.ratings.tsv"
spark.sparkContext.addFile(url)
movie_ratings = spark.read.csv(SparkFiles.get("title.ratings.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
movie_ratings.show(10)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.6|    1550|
|tt0000002|          6.1|     186|
|tt0000003|          6.5|    1207|
|tt0000004|          6.2|     113|
|tt0000005|          6.1|    1934|
|tt0000006|          5.2|     102|
|tt0000007|          5.5|     615|
|tt0000008|          5.4|    1668|
|tt0000009|          5.4|      81|
|tt0000010|          6.9|    5549|
+---------+-------------+--------+
only showing top 10 rows



In [9]:
# Count DataFrame
movie_ratings.count()

993821

In [10]:
# Read in title_crew data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/title.crew.tsv"
spark.sparkContext.addFile(url)
movie_crew = spark.read.csv(SparkFiles.get("title.crew.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
movie_crew.show(10)

+---------+-------------------+---------+
|   tconst|          directors|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|       \N|
|tt0000002|          nm0721526|       \N|
|tt0000003|          nm0721526|       \N|
|tt0000004|          nm0721526|       \N|
|tt0000005|          nm0005690|       \N|
|tt0000006|          nm0005690|       \N|
|tt0000007|nm0374658,nm0005690|       \N|
|tt0000008|          nm0005690|       \N|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|       \N|
+---------+-------------------+---------+
only showing top 10 rows



In [11]:
# Count DataFrame
movie_crew.count()

7232795

In [ ]:
# drop duplicate reviews

In [ ]:
# Drop null values
# cleaned_df = df.dropna()
# cleaned_df.show()

### Clean Data

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only required columns from each data set
# cleaned_df.filter(col("col1","col2") == True)
# cleaned_df.show()

In [15]:
movie_data = pd.merge(movie_akas_2, movie_basics, on=["tconst", "tconst"])

NameError: ignored

### Connection to Postgres

In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [13]:
# Configure settings for RDS 
mode = "append"
jdbc_url="jdbc:postgresql://movies-dataviz.cv7p5opuktsx.us-west-1.rds.amazonaws.com:5432/movie_analysis_db"
config = {"user":"postgres",
          "password":"20202020",
          "driver":"org.postgresql.Driver"}

### Write DataFrames to tables in RDS

In [ ]:
movie_akas_us.write.jdbc(url=jdbc_url, table='movie_akas_us', mode=mode, properties=config)

In [17]:
movie_basics.write.jdbc(url=jdbc_url, table='movie_basics', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
movie_ratings.write.jdbc(url=jdbc_url, table='movie_ratings', mode=mode, properties=config)

In [ ]:
movie_crew.write.jdbc(url=jdbc_url, table='movie_crew', mode=mode, properties=config)

### # Join the tables

In [ ]:
# vine_table_df = review_data_df.join(vine_data_df, on="review_id", how="inner")
# vine_table_df.show()

In [ ]:
# Merging dataframes in Pandas

movie_data = pd.merge(movie_akas_2, movie_basics, on=["tconst", "tconst"])
movie_data = pd.merge(movie_data, movie_crew, on=["tconst", "tconst"])
movie_data = pd.merge(movie_data, movie_ratings, on=["tconst", "tconst"])

movie_data

In [ ]:
# merge tables on left joint
movie_akas = movie_basics.join(movie_akas, on="tconst", how="left")
movie_akas = movie_crew.join(movie_akas, on="tconst", how="left")
movie_akas = movie_ratings.join(movie_akas, on="tconst", how="left")

movie_akas